In [1]:
import os

#USE ONLY ONE OF THESE:
# os.environ["MODIN_ENGINE"] = "ray"  # Modin will use Ray
os.environ["MODIN_ENGINE"] = "dask"  # Modin will use Dask

import modin.pandas as pd
# import pandas as pd
from pathlib import Path
from tqdm import tqdm


In [2]:
# input
data_dir = Path("/Users/oholm/annad/ordtidni/Gigaword/output/")
input_file_path = data_dir.glob('*/**/*')
filepaths = [item for item in input_file_path if (item.is_file() and str(item).endswith('.parquet'))]
# output
output_filepath = data_dir/'word_occurrence_counts.parquet'

In [3]:
%%time
print('about to process', len(filepaths), 'files')
word_occurance_count = pd.DataFrame({'occurance_count':[0]}, index=['barn']) # cold start dataframe
# display(word_occurance_count)
print('----'*8, 'start iteration over files', '----'*8)
for filepath in tqdm(filepaths):
#     print(filepath.parent)
    corpus_doc = pd.read_parquet(filepath)
#     print('read in dataframe')
#     display(corpus_doc.head(2))
    noun_mask = corpus_doc['POS'].str.startswith('n')
#     print('there are', sum(noun_mask), 'nouns in the set')
    nouns_only_df = corpus_doc[noun_mask]
#     print('nouns only')
#     display(nouns_only_df.sample(3))
#     all_nouns_list = nouns_only_df['Lemma'].unique()
    noun_occurrence_count =\
        nouns_only_df[['Lemma', 'POS']]\
        .groupby(by='Lemma')\
        .count()\
        ['POS']\
        .rename('additions')\
        
    word_occurance_count = pd.concat([word_occurance_count, noun_occurrence_count], axis=1).fillna(0)
    word_occurance_count['occurance_count'] = word_occurance_count['occurance_count'] + word_occurance_count['additions']
    word_occurance_count.drop('additions', axis=1,inplace=True)

#     print('there are', len(word_occurance_count), 'noun lemmas')

output_filepath = data_dir/'word_occurrence_counts.parquet'
word_occurance_count.to_parquet(output_filepath)

about to process 661 files


  0%|          | 0/661 [00:00<?, ?it/s]

-------------------------------- start iteration over files --------------------------------


  1%|          | 4/661 [00:11<32:11,  2.94s/it]UserWarning: Large object of size 1.03 MB detected in task graph: 
  (0, <function BasePandasFrame._copartition.<locals ... -e04305c39856')
Consider scattering large objects ahead of time
with client.scatter to reduce scheduler burden and 
keep data on workers

    future = client.submit(func, big_data)    # bad

    big_future = client.scatter(big_data)     # good
    future = client.submit(func, big_future)  # good
 35%|███▍      | 230/661 [34:53<1:26:14, 12.00s/it]

Exception: Internal Error. Please email bug_reports@modin.org with the traceback and command that caused this error.
Row lengths: 224 != 90

```
  0%|          | 0/661 [00:00<?, ?it/s]
about to process 661 files
-------------------------------- start iteration over files --------------------------------
100%|██████████| 661/661 [50:02<00:00,  4.54s/it]
CPU times: user 48min 51s, sys: 5min 49s, total: 54min 40s
Wall time: 50min 5s
```

In [4]:
word_occurance_count.shape

(948492, 1)

In [5]:
top10_nouns_freq = word_occurance_count.sort_values(by='occurance_count', ascending=False).head(60).reset_index()
top10_nouns_freq

,index,occurance_count
0,mál,1455991.0
1,ár,1053546.0
2,maður,834866.0
3,lag,802521.0
4,forseti,777138.0
5,land,536738.0
6,stefnandi,504338.0
7,vegur,504154.0
8,tími,387157.0
9,dagur,382353.0


In [6]:
print('there are', sum(word_occurance_count['occurance_count'] < 2.0), 'single occurence lemmas in corpus')

there are 529960 single occurence lemmas in corpus
